In [19]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 200)

# ---- paths (edit these) ----
FL_PATH  = "../Data/Flight_Level_Data.csv"
PNR_PATH = "../Data/PNR_Flight_Level_Data.csv"
BAG_PATH = "../Data/Bag_Level_Data.csv"
REM_PATH = "../Data/PNR_Remark_Level_Data.csv"
APT_PATH = "../Data/Airports_Data.csv"


In [2]:
FLIGHT_KEYS = ["company_id", "flight_number", "scheduled_departure_date_local"]

def clean_str(s):
    return (s.astype(str)
             .str.strip()
             .str.upper()
             .replace({"NAN": np.nan}))


In [3]:
usecols_fl = [
    "company_id","flight_number",
    "scheduled_departure_date_local",
    "scheduled_departure_station_code","scheduled_arrival_station_code",
    "scheduled_departure_datetime_local","scheduled_arrival_datetime_local",
    "actual_departure_datetime_local","actual_arrival_datetime_local",
    "scheduled_ground_time_minutes","actual_ground_time_minutes","minimum_turn_minutes",
    "total_seats","fleet_type","carrier"
]

fl = pd.read_csv(
    FL_PATH, usecols=usecols_fl,
    parse_dates=[
        "scheduled_departure_datetime_local","scheduled_arrival_datetime_local",
        "actual_departure_datetime_local","actual_arrival_datetime_local"
])
# standardize keys
fl["company_id"] = clean_str(fl["company_id"])
fl["flight_number"] = clean_str(fl["flight_number"])
fl["scheduled_departure_date_local"] = pd.to_datetime(fl["scheduled_departure_date_local"]).dt.date
fl["scheduled_departure_date_local"] = pd.to_datetime(fl["scheduled_departure_date_local"])

# stations as categorical (saves RAM)
for c in ["scheduled_departure_station_code","scheduled_arrival_station_code","fleet_type","carrier"]:
    if c in fl:
        fl[c] = clean_str(fl[c]).astype("category")

# --- engineer delay and buffers
fl["departure_delay_minutes"] = (
    (fl["actual_departure_datetime_local"] - fl["scheduled_departure_datetime_local"])
    .dt.total_seconds() / 60
)

fl["arrival_delay_minutes"] = (
    (fl["actual_arrival_datetime_local"] - fl["scheduled_arrival_datetime_local"])
    .dt.total_seconds() / 60
)

fl["ground_time_buffer"] = fl["scheduled_ground_time_minutes"] - fl["minimum_turn_minutes"]

# Make a unique flight id (handy later)
fl["flight_id"] = (
    fl["company_id"] + "_" +
    fl["flight_number"] + "_" +
    fl["scheduled_departure_date_local"].dt.strftime("%Y-%m-%d")
)


In [6]:
# remove exact duplicates first
fl = fl.drop_duplicates()

# if still multiple, keep the latest actual departure record
fl = fl.sort_values("actual_departure_datetime_local").drop_duplicates(FLIGHT_KEYS, keep="last")

assert fl[FLIGHT_KEYS].drop_duplicates().shape[0] == fl.shape[0], "Flight keys not unique!"
fl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8063 entries, 3196 to 6322
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   company_id                          8063 non-null   object             
 1   flight_number                       8063 non-null   object             
 2   scheduled_departure_date_local      8063 non-null   datetime64[ns]     
 3   scheduled_departure_station_code    8063 non-null   category           
 4   scheduled_arrival_station_code      8063 non-null   category           
 5   scheduled_departure_datetime_local  8063 non-null   datetime64[ns, UTC]
 6   scheduled_arrival_datetime_local    8063 non-null   datetime64[ns, UTC]
 7   actual_departure_datetime_local     8063 non-null   datetime64[ns, UTC]
 8   actual_arrival_datetime_local       8063 non-null   datetime64[ns, UTC]
 9   total_seats                         8063 no

In [ ]:
usecols_pnr = [
    "record_locator", "company_id","flight_number","scheduled_departure_date_local",
    "total_pax","lap_child_count","is_child","basic_economy_ind","is_stroller_user"
]

pnr = pd.read_csv(PNR_PATH, usecols=usecols_pnr)

# clean up
pnr["company_id"] = clean_str(pnr["company_id"])
pnr["flight_number"] = clean_str(pnr["flight_number"])
pnr["scheduled_departure_date_local"] = pd.to_datetime(pnr["scheduled_departure_date_local"])

# 1) Load remarks (no scheduled_departure_date_local here!)
usecols_rem = ["record_locator","pnr_creation_date","flight_number","special_service_request"]
rem = pd.read_csv(REM_PATH, usecols=usecols_rem)

# clean
rem["flight_number"] = clean_str(rem["flight_number"])
rem["special_service_request"] = clean_str(rem["special_service_request"])

# 2) Attach flight info from PNR (this is where we get company_id and date)
pnr_keys = pnr[["record_locator","flight_number","company_id","scheduled_departure_date_local"]].drop_duplicates()

rem = rem.merge(
    pnr_keys,
    on=["record_locator","flight_number"],
    how="left"
)

# 3) Engineer SSR flags
rem["is_wheelchair"] = rem["special_service_request"].fillna("").str.contains(r"\bWCH[RS C]\b|WHEELCHAIR", regex=True).astype(int)
rem["is_special_seat"] = rem["special_service_request"].fillna("").str.contains("SEAT|ASSIST", regex=True).astype(int)

# 4) Aggregate to flight level
rem_agg = (
    rem.groupby(["company_id","flight_number","scheduled_departure_date_local"], as_index=False)
       .agg(ssr_count=("special_service_request","count"),
            wheelchair_reqs=("is_wheelchair","sum"),
            special_seat_reqs=("is_special_seat","sum"))
)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51698 entries, 0 to 51697
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   record_locator                  51698 non-null  object        
 1   pnr_creation_date               51698 non-null  object        
 2   flight_number                   51698 non-null  object        
 3   special_service_request         51698 non-null  object        
 4   company_id                      17067 non-null  object        
 5   scheduled_departure_date_local  17067 non-null  datetime64[ns]
 6   is_wheelchair                   51698 non-null  int64         
 7   is_special_seat                 51698 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 3.2+ MB


In [32]:
usecols_bag = [
    "company_id","flight_number","scheduled_departure_date_local",
    "bag_tag_unique_number","bag_type"
]
bag = pd.read_csv(BAG_PATH, usecols=usecols_bag)

bag["company_id"] = clean_str(bag["company_id"])
bag["flight_number"] = clean_str(bag["flight_number"])
bag["scheduled_departure_date_local"] = pd.to_datetime(bag["scheduled_departure_date_local"])
bag["bag_type"] = clean_str(bag["bag_type"])

# derive bag flags
# Flag transfer & hot transfer
bag["is_transfer"] = bag["bag_type"].isin(["TRANSFER","HOT TRANSFER","HOT_TRANSFER"]).astype(int)

# Flag origin bags
bag["is_origin"] = (bag["bag_type"] == "ORIGIN").astype(int)

# Aggregate at flight level
bag_agg = (
    bag.groupby(FLIGHT_KEYS, as_index=False)
       .agg(total_bags=("bag_tag_unique_number","nunique"),
            transfer_bags=("is_transfer","sum"),
            origin_bags=("is_origin","sum"))
)

# New ratio: (transfer + hot transfer) / origin
bag_agg["bag_ratio"] = np.where(bag_agg["origin_bags"] > 0,
                                bag_agg["transfer_bags"] / bag_agg["origin_bags"],
                                np.nan)



In [33]:
apt = pd.read_csv(APT_PATH)
apt["airport_iata_code"] = clean_str(apt["airport_iata_code"])
apt["iso_country_code"] = clean_str(apt["iso_country_code"])

# join twice: dep & arr
fl = fl.merge(apt[["airport_iata_code","iso_country_code"]].rename(columns={
    "airport_iata_code":"scheduled_departure_station_code",
    "iso_country_code":"dep_country"
}), on="scheduled_departure_station_code", how="left")

fl = fl.merge(apt[["airport_iata_code","iso_country_code"]].rename(columns={
    "airport_iata_code":"scheduled_arrival_station_code",
    "iso_country_code":"arr_country"
}), on="scheduled_arrival_station_code", how="left")

fl["is_international"] = (fl["dep_country"] != fl["arr_country"]).astype("int8")


KeyError: 'dep_country'

In [34]:
base = fl.copy()

for (df, name) in [(pnr_agg, "pnr_agg"), (bag_agg, "bag_agg"), (rem_agg, "rem_agg")]:
    before = base.shape[0]
    base = base.merge(df, on=FLIGHT_KEYS, how="left")
    after = base.shape[0]
    assert after == before, f"{name} merge changed row count! Check keys."

# post-merge feature engineering
base["load_factor"] = np.where(base["total_seats"].gt(0),
                               base["total_pax"] / base["total_seats"], np.nan)

# defensive fills (use with care; keep raw nulls if you want)
for c in ["ssr_count","wheelchair_reqs","special_seat_reqs",
          "total_bags","transfer_bags","bag_ratio",
          "lap_child_count","child_pax","basic_economy_ind","stroller_users"]:
    if c in base:
        base[c] = base[c].fillna(0)


In [35]:
# duplicates?
dups = base[FLIGHT_KEYS].duplicated().sum()
print("Duplicate key rows:", dups)

# missing key parts
print("Missing keys per column:\n", base[FLIGHT_KEYS].isna().sum())

# quick stats
print(base[["departure_delay_minutes","arrival_delay_minutes",
            "ground_time_buffer","load_factor","bag_ratio"]].describe().T)

# delayed share
pct_delayed = (base["departure_delay_minutes"]>0).mean()
print(f"% flights departing late: {pct_delayed:.1%}")


Duplicate key rows: 0
Missing keys per column:
 company_id                        0
flight_number                     0
scheduled_departure_date_local    0
dtype: int64
                          count        mean         std      min    25%  \
departure_delay_minutes  8063.0   21.218033   62.185481   -17.00  -4.00   
arrival_delay_minutes    8063.0   15.459878   64.217776   -54.00 -14.00   
ground_time_buffer       8063.0  136.069081  563.989908 -1009.00  14.00   
load_factor              8063.0    1.024249    0.209811     0.06   0.94   
bag_ratio                8063.0    3.480683    6.451044     0.00   0.70   

                               50%        75%        max  
departure_delay_minutes   0.000000  18.000000   1017.000  
arrival_delay_minutes    -3.000000  18.000000   1001.000  
ground_time_buffer       29.000000  64.000000  33281.000  
load_factor               1.047619   1.145251      1.895  
bag_ratio                 1.562500   3.328431     99.000  
% flights departing late: 

In [36]:
# Load factor: clip to [0,1.2]
base["load_factor"] = base["load_factor"].clip(0, 1.2)

# Bag ratio: clip to [0,1]
base["bag_ratio"] = base["bag_ratio"].clip(0,1)

# Ground time buffer: remove extreme outliers
base["ground_time_buffer"] = base["ground_time_buffer"].clip(-200, 500)  # adjust thresholds if needed


In [37]:
base["is_departure_delayed"] = (base["departure_delay_minutes"] > 0).astype(int)
base["is_arrival_delayed"]   = (base["arrival_delay_minutes"] > 0).astype(int)


In [38]:
base[["load_factor","bag_ratio","ground_time_buffer"]].describe().T


,count,mean,std,min,25%,50%,75%,max
load_factor,8063.0,1.008428,0.187335,0.06,0.94,1.047619,1.145251,1.2
bag_ratio,8063.0,0.823838,0.293094,0.00,0.70,1.000000,1.000000,1.0
ground_time_buffer,8063.0,84.001984,150.329012,-200.00,14.00,29.000000,64.000000,500.0


In [39]:
# Best option
base.to_parquet("../Data/flights_clean.parquet", index=False)
